# Asynchronous Programming in Python: A Comprehensive Guide

This notebook provides a step-by-step guide to understanding and implementing asynchronous programming in Python using `async/await`. We'll cover everything from basic concepts to real-world applications.

## What is Asynchronous Programming?

Asynchronous programming is a programming paradigm that allows concurrent execution of multiple tasks without blocking the main program. In Python, this is achieved using coroutines, which are special functions that can pause their execution while waiting for operations to complete.

Key concepts we'll cover:
1. Coroutines with async/await
2. Event loops
3. Tasks and concurrent execution
4. Async context managers
5. Error handling
6. Real-world applications

## 1. Setting Up Async Environment

First, let's set up our environment and understand the basics. We'll need the `asyncio` module, which is Python's standard library for writing asynchronous code using coroutines.

In [ ]:
import asyncio
import time

# A simple async function
async def say_hello(delay, message):
    await asyncio.sleep(delay)  # Non-blocking sleep
    print(f"{message} (after {delay} seconds)")

# Running a single coroutine
async def main():
    print("Start")
    await say_hello(1, "Hello, Async World!")
    print("End")

# Run the async function
print("Running async function...")
asyncio.run(main())

## 2. Basic Async/Await Example

Now let's explore a more practical example that demonstrates the power of async programming. We'll create multiple coroutines and see how they can run concurrently.

In [ ]:
async def process_item(item, delay):
    print(f"Starting to process {item}")
    await asyncio.sleep(delay)  # Simulate some async work
    print(f"Finished processing {item}")
    return f"Processed {item}"

async def main():
    # Create multiple coroutines
    start_time = time.time()
    
    tasks = [
        process_item("Item 1", 2),
        process_item("Item 2", 1),
        process_item("Item 3", 3)
    ]
    
    # Run coroutines concurrently and gather results
    results = await asyncio.gather(*tasks)
    
    end_time = time.time()
    print(f"\nAll items processed in {end_time - start_time:.2f} seconds")
    print("Results:", results)

# Run the async function
asyncio.run(main())

## 3. Working with Multiple Tasks

Tasks are a way to schedule coroutines to run independently. Let's see how to create and manage multiple tasks using `asyncio.create_task()`.

In [ ]:
async def long_running_task(name, duration):
    print(f"Task {name} starting...")
    try:
        await asyncio.sleep(duration)
        print(f"Task {name} completed after {duration} seconds")
        return f"Result from {name}"
    except asyncio.CancelledError:
        print(f"Task {name} was cancelled!")
        raise

async def main():
    # Create tasks
    task1 = asyncio.create_task(long_running_task("A", 3))
    task2 = asyncio.create_task(long_running_task("B", 2))
    
    # Wait for both tasks to complete
    try:
        results = await asyncio.gather(task1, task2)
        print("All tasks completed:", results)
    except asyncio.CancelledError:
        print("Some tasks were cancelled!")

# Run the async function
asyncio.run(main())

## 4. Async Context Managers

Async context managers are useful for managing resources that need to be acquired and released asynchronously. Here's an example using an async context manager for a simulated database connection.

In [ ]:
class AsyncDatabaseConnection:
    async def __aenter__(self):
        # Simulate connecting to a database
        print("Connecting to database...")
        await asyncio.sleep(1)
        print("Connected!")
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        # Simulate closing the connection
        print("Closing connection...")
        await asyncio.sleep(0.5)
        print("Connection closed!")
    
    async def query(self, query):
        # Simulate a database query
        print(f"Executing query: {query}")
        await asyncio.sleep(1)
        return [f"Result for {query}"]

async def main():
    async with AsyncDatabaseConnection() as db:
        result = await db.query("SELECT * FROM users")
        print("Query result:", result)

# Run the example
asyncio.run(main())

## 5. Error Handling in Async Code

Error handling in async code is similar to synchronous code, but there are some special considerations for handling timeouts and cancellations.

In [ ]:
async def risky_operation(timeout):
    try:
        print("Starting risky operation...")
        await asyncio.sleep(timeout)
        print("Operation completed successfully!")
        return "Success"
    except asyncio.TimeoutError:
        print("Operation timed out!")
        raise
    except asyncio.CancelledError:
        print("Operation was cancelled!")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

async def main():
    try:
        # Try to run the operation with a timeout
        result = await asyncio.wait_for(risky_operation(5), timeout=3)
        print("Result:", result)
    except asyncio.TimeoutError:
        print("The operation took too long!")
    except Exception as e:
        print(f"Caught an error: {e}")

# Run the example
asyncio.run(main())

## 6. Real-world Example: Async Web Requests

Let's create a practical example using `aiohttp` to perform concurrent web requests. First, we need to install the `aiohttp` package.

In [ ]:
# Install aiohttp if not already installed
import sys
import subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'aiohttp'])

In [ ]:
import aiohttp
import asyncio
import time

async def fetch_url(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    # List of URLs to fetch
    urls = [
        'https://api.github.com/events',
        'https://api.github.com/repos/python/cpython',
        'https://api.github.com/repos/pallets/flask'
    ]
    
    start_time = time.time()
    
    async with aiohttp.ClientSession() as session:
        # Create tasks for each URL
        tasks = [fetch_url(session, url) for url in urls]
        
        # Gather all results
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Process results
        for url, result in zip(urls, results):
            if isinstance(result, Exception):
                print(f"Error fetching {url}: {result}")
            else:
                print(f"Successfully fetched {url}, length: {len(result)} characters")
    
    end_time = time.time()
    print(f"\nFetched {len(urls)} URLs in {end_time - start_time:.2f} seconds")

# Run the example
asyncio.run(main())

## Conclusion

This notebook has demonstrated the key concepts of asynchronous programming in Python:

1. Basic async/await syntax and coroutines
2. Running multiple tasks concurrently
3. Using async context managers
4. Handling errors in async code
5. Real-world application with web requests

Async programming is particularly useful for I/O-bound tasks like:
- Web scraping
- API calls
- Database operations
- File operations

Remember that async programming is not suitable for CPU-bound tasks - for those, you should use multiprocessing instead.